# Credit Risk Resampling Techniques

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import ClusterCentroids
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier 


# Read the CSV and Perform Basic Data Cleaning

In [2]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [3]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df = df.loc[:, columns].copy()

#Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

C:\Users\hbyrd\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,17.19%,375.35,RENT,66000.0,Source Verified,Mar-19,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,20.00%,929.09,MORTGAGE,105000.0,Verified,Mar-19,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,20.00%,529.88,MORTGAGE,56000.0,Verified,Mar-19,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,16.40%,353.55,RENT,92000.0,Verified,Mar-19,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,14.74%,520.39,MORTGAGE,52000.0,Not Verified,Mar-19,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [4]:
# Label Encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# "home_ownership"
label_encoder.fit(df["home_ownership"])
df["homeownership_le"] = label_encoder.transform(df["home_ownership"])
df=df.drop(columns=['home_ownership'])

# "verification_status"
label_encoder.fit(df["verification_status"])
df["verificationstatus_le"] = label_encoder.transform(df["verification_status"])
df=df.drop(columns=['verification_status'])

# "issue_d"
months_num = {
    "Jan": 1,
   "Ja": 1,
    "Feb": 2,
    "Fe": 2,
   "Mar": 3,
    "Ma": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12,
}

df['issue_month'] = df['issue_d'].str[0:2]
df["month_num"] = df["issue_month"].apply(lambda x: months_num[x])
df = df.drop(columns=["issue_d","issue_month"])

# "pymnt_plan"
label_encoder.fit(df["pymnt_plan"])
df["pymnt_plan_le"] = label_encoder.transform(df["pymnt_plan"])
df=df.drop(columns=['pymnt_plan'])

# "hardship_flag"
label_encoder.fit(df["hardship_flag"])
df["hardship_flag_le"] = label_encoder.transform(df["hardship_flag"])
df=df.drop(columns=['hardship_flag'])

# "debt_settlement_flag"
label_encoder.fit(df["debt_settlement_flag"])
df["debt_settlement_flag_le"] = label_encoder.transform(df["debt_settlement_flag"])
df=df.drop(columns=['debt_settlement_flag'])

# "initial_list_status"
label_encoder.fit(df["initial_list_status"])
df["initial_list_status"] = label_encoder.transform(df["initial_list_status"])
df=df.drop(columns=['initial_list_status'])

df=df.drop(columns=['next_pymnt_d'])

# "application_type"
label_encoder.fit(df["application_type"])
df["application_type_le"] = label_encoder.transform(df["application_type"])
df=df.drop(columns=['application_type'])

# int_rate
#df['int_rate'] = df['int_rate'].append(float(x.strip('%'))/100)

cleancol = []

# for index tracking
counter = 0

#column iteration
for x in df['int_rate']:
    #finding % values
    if df['int_rate'][counter].endswith('%') == True:
        cleancol.append(float(x.strip('%'))/100)
    else:
        break
    #keep up with index
    counter += 1

df['int_rate']=cleancol

df.head(10)

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,homeownership_le,verificationstatus_le,month_num,pymnt_plan_le,hardship_flag_le,debt_settlement_flag_le,application_type_le
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,38199.0,2000.0,61987.0,3,1,3,0,0,0,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,60641.0,41200.0,49197.0,1,2,3,0,0,0,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,45684.0,7500.0,43144.0,1,2,3,0,0,0,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,68784.0,19700.0,76506.0,3,2,3,0,0,0,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,25919.0,27600.0,20000.0,1,0,3,0,0,0,0
5,25000.0,0.1797,634.43,90000.0,low_risk,18.49,1.0,1.0,14.0,0.0,...,418934.0,18800.0,369051.0,1,1,3,0,0,0,0
6,20400.0,0.2000,540.48,51315.0,low_risk,25.63,0.0,2.0,8.0,1.0,...,23469.0,3500.0,27523.0,3,1,3,0,0,0,0
7,10000.0,0.1557,349.46,66000.0,low_risk,9.00,0.0,0.0,11.0,0.0,...,22377.0,8400.0,12500.0,2,1,3,0,0,0,0
8,14000.0,0.0881,443.96,45000.0,low_risk,10.24,0.0,1.0,9.0,0.0,...,9724.0,12100.0,3000.0,3,0,3,0,0,0,0
9,10000.0,0.1474,345.39,63295.0,low_risk,39.34,0.0,0.0,27.0,0.0,...,102924.0,15300.0,80722.0,3,2,3,0,0,0,0


# Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns = 'loan_status')

# Create our target
y = df['loan_status']

Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,homeownership_le,verificationstatus_le,month_num,pymnt_plan_le,hardship_flag_le,debt_settlement_flag_le,application_type_le
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,6.133843e+04,29734.128558,5.572240e+04,1.812779,0.669994,1.726172,0.0,0.0,0.0,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,5.738798e+04,26795.394232,5.095845e+04,0.941313,0.719105,0.743862,0.0,0.0,0.0,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,2.350000e+02,100.000000,1.270000e+02,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,2.650300e+04,11600.000000,2.288000e+04,1.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,4.535700e+04,22100.000000,4.200000e+04,1.000000,1.000000,2.000000,0.0,0.0,0.0,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,7.657000e+04,39300.000000,7.249900e+04,3.000000,1.000000,2.000000,0.0,0.0,0.0,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.295455e+06,509400.000000,1.426964e+06,3.000000,2.000000,3.000000,0.0,0.0,0.0,1.000000


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = scaler.fit(X_train)


In [11]:
# Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOverSampler

X_resampled, y_resampled = RandomOverSampler(random_state=1).fit_resample(X_train_scaled, y_train)

Counter(y_resampled)


Counter({'low_risk': 51366, 'high_risk': 51366})

In [13]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test_scaled)

C:\Users\hbyrd\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
# Calculated the balanced accuracy score

balanced_accuracy_score(y_test, y_pred)

0.7682118246904204

In [15]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[   75,    26],
       [ 3526, 13578]], dtype=int64)

In [16]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.74      0.79      0.04      0.77      0.59       101
   low_risk       1.00      0.79      0.74      0.88      0.77      0.59     17104

avg / total       0.99      0.79      0.74      0.88      0.77      0.59     17205



### SMOTE Oversampling

In [17]:
# Resample the training data with SMOTE

X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(X_train_scaled, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [18]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test_scaled)

C:\Users\hbyrd\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7763487088886625

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   76,    25],
       [ 3417, 13687]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.75      0.80      0.04      0.78      0.60       101
   low_risk       1.00      0.80      0.75      0.89      0.78      0.61     17104

avg / total       0.99      0.80      0.75      0.88      0.78      0.60     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [22]:
# Resample the data using the ClusterCentroids resampler

X_resampled, y_resampled = ClusterCentroids(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [23]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test_scaled)

C:\Users\hbyrd\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7495736623475258

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   82,    19],
       [ 5349, 11755]], dtype=int64)

In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.81      0.69      0.03      0.75      0.56       101
   low_risk       1.00      0.69      0.81      0.81      0.75      0.55     17104

avg / total       0.99      0.69      0.81      0.81      0.75      0.55     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [42]:
# Resample the training data with SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 51361, 'low_risk': 46653})

In [29]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test_scaled)

C:\Users\hbyrd\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.611206399522085

In [31]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   87,    14],
       [10929,  6175]], dtype=int64)

In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.86      0.36      0.02      0.56      0.33       101
   low_risk       1.00      0.36      0.86      0.53      0.56      0.30     17104

avg / total       0.99      0.36      0.86      0.53      0.56      0.30     17205



**Questions answered in seperate markdown file.**

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [33]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)
y_pred_rf = brf.predict(X_test) 

In [34]:
# Calculated the balanced accuracy score

balanced_accuracy_score(y_test, y_pred_rf)

0.7515444247885967

In [35]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred_rf)

array([[   63,    38],
       [ 2064, 15040]], dtype=int64)

In [36]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.62      0.88      0.06      0.74      0.53       101
   low_risk       1.00      0.88      0.62      0.93      0.74      0.56     17104

avg / total       0.99      0.88      0.63      0.93      0.74      0.56     17205



In [37]:
# List the features sorted in descending order by feature importance

importances = brf.feature_importances_

importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.06607490072728682, 'total_rec_prncp'),
 (0.06356072282512583, 'total_rec_int'),
 (0.06313117434543672, 'total_pymnt'),
 (0.060730609428838296, 'total_pymnt_inv'),
 (0.05399266686407467, 'last_pymnt_amnt'),
 (0.03353780574781485, 'int_rate'),
 (0.032277471900396926, 'month_num'),
 (0.02038661655587231, 'mths_since_recent_inq'),
 (0.02022237823990287, 'dti'),
 (0.01817643945661585, 'installment')]

### Easy Ensemble Classifier

In [38]:
# Train the EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=1)
eec.fit(X_train_scaled, y_train) 
y_pred_eec = eec.predict(X_test) 

In [39]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred_eec)

0.5248717224388482

In [40]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_eec)

array([[   11,    90],
       [ 1012, 16092]], dtype=int64)

In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.11      0.94      0.02      0.32      0.09       101
   low_risk       0.99      0.94      0.11      0.97      0.32      0.11     17104

avg / total       0.99      0.94      0.11      0.96      0.32      0.11     17205



**Questions answered in seperate markdown file.**